In [ ]:
pip install boto3 pandas pyarrow ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=b742cb1d1b2386a1893c00fa620aabf88cf8a477e9a1b5f23ee7ef6dd0805053
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


# **1 - Importar Bibliotecas Necessárias**

In [ ]:
# Para vizualizar e manipular os dados
import pandas as pd
import numpy as np
import seaborn as sns
import boto3
import pyarrow.parquet as pq
from io import BytesIO
import configparser
import os

# Para gerar indicadores técnicos
import ta

# Para executar os modelos
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

# Para gerar as métricas de performance
from sklearn.metrics import classification_report, accuracy_score

# **2 - Carregar Lista de Ações**

In [ ]:
# Configurar suas credenciais manualmente
aws_access_key = os.getenv('AWS_ACCESS_KEY')
aws_secret_key = os.getenv('AWS_SECRET_KEY')
session_token = os.getenv('AWS_SESSION_TOKEN')

# Iniciar uma sessão com as credenciais
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    aws_session_token=session_token  # Inclua isso se houver um token de sessão
)

# Nome do bucket S3 e o prefixo (caminho dentro do bucket)
bucket_name = 'techchanllange-fiap2024'
prefix = 'parquet/'  # Pode ser uma pasta ou subdiretório no bucket

# Listar todos os arquivos parquet dentro do bucket/pasta
response = s3.list_objects_v2(Bucket=bucket_name, Prefix = prefix)
arquivos = [content['Key'] for content in response.get('Contents', [])]

# Lista para armazenar os dataframes
dataframes = []

# Ler cada arquivo parquet e adicionar à lista de dataframes
for arquivo in arquivos:
  obj = s3.get_object(Bucket=bucket_name, Key=arquivo)
  # Verifique o tamanho do arquivo antes de tentar ler
  if obj['ContentLength'] > 0:
    with BytesIO(obj['Body'].read()) as f:
      df = pq.read_table(f).to_pandas()
      dataframes.append(df)

# Consolidar todos os dataframes em um único dataframe
dados_full = pd.concat(dataframes, ignore_index=True)

# Exibir o dataframe consolidado
dados_full.shape

(298009, 113)

# **3 - Calcular indicadores técnicos**

In [ ]:
# Função para calcular indicadores técnicos
def calculate_indicators(df):
    dados_full['SMA_10'] = ta.trend.sma_indicator(dados_full['Fechamento'], window=10) # Média Móvel Dos Últimos 10 Dias
    dados_full['EMA_10'] = ta.trend.ema_indicator(dados_full['Fechamento'], window=10) # Média Móvel Exponencial
    dados_full['RSI'] = ta.momentum.rsi(dados_full['Fechamento'], window=14) # Índice de Força Relativa
    williams_r = ta.momentum.WilliamsRIndicator(high=dados_full['MaiorValor'], low=dados_full['MenorValor'], close=dados_full['Fechamento'], lbp=14)
    dados_full['Williams_%R'] = williams_r.williams_r() # Williams %R - Oscilador de Momentum
    dados_full['ROC'] = ta.momentum.roc(dados_full['Fechamento'], window=12) # Taxa de Variação
    dados_full['MACD'] = ta.trend.macd(dados_full['Fechamento']) # Diferença Entre EMA de Curto e Longo Prazo
    dados_full['MACD_signal'] = ta.trend.macd_signal(dados_full['Fechamento']) # Linha de Curto Prazo do MACD
    dados_full['bollinger_hband'] = ta.volatility.bollinger_hband(dados_full['Fechamento'], window=20, window_dev=2) # Banda de Bolliger High
    dados_full['bollinger_lband'] = ta.volatility.bollinger_lband(dados_full['Fechamento'], window=20, window_dev=2) # Banda de Bolliger Low

# **4 - Calcular a Variável Target**

In [ ]:
# Função para calcular a tendência (1: Alta, 0: Neutra, -1: Baixa) por ticker
def calcular_tendencia_por_ticker(df):

    # Agrupar os dados por Ticker e aplicar a função de cálculo da tendência para cada grupo
    dados_full['Percentual'] = dados_full.groupby('Ticker')['Fechamento'].pct_change()  # Calcula o retorno percentual dentro de cada ticker

    # Condições para definir alta, neutra ou baixa
    condicoes = [
        (dados_full['Percentual'] > 0.01),  # Alta (ajuste o valor conforme necessário)
        (dados_full['Percentual'] < -0.01)  # Baixa (ajuste o valor conforme necessário)
    ]
    escolhas = [1, -1]

    # Aplicar as condições e definir a tendência
    dados_full['Tendencia'] = np.select(condicoes, escolhas, default=0)  # Neutro

    return dados_full

# **5 - Aplicar Cálculos a Base**

In [ ]:
# Ordenando por ação e data, garantindo que os dados de cada ação estejam em ordem cronológica
dados_full.sort_values(by=['Ticker', 'Data'], inplace=True)

In [ ]:
# Aplicar os cálculos para cada ticker
dados_full.groupby('Ticker').apply(calculate_indicators)


In [ ]:
# Aplicar a função para calcular a tendência
dados_full = calcular_tendencia_por_ticker(dados_full)

<ipython-input-58-9d9533e3d4a5>:5: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  dados_full['Percentual'] = dados_full.groupby('Ticker')['Fechamento'].pct_change()  # Calcula o retorno percentual dentro de cada ticker


# **6 - Análise Exploratória dos Dados**

In [ ]:
# Ajustar o Pandas para mostrar todas as colunas e linhas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Exibir Amostra da Tabela
dados_full.sample(5)

,Ticker,Data,Abertura,MenorValor,MaiorValor,Fechamento,Volume,Dividendos,Desdobramento,Cidade,Estado,Industria,ChaveIndustria,DescricaoIndustria,Setor,ChaveSetor,DescricaoSetor,EmpregadosEmTempoIntegral,RiscoDeAuditoria,RiscoDoConselho,RiscoDeCompensacao,RiscoDeDireitosDeAcionistas,RiscoGeral,DataDaEraDeGovernanca,IdadeMaxima,DicaDePreco,FechamentoAnterior,FechamentoAnteriorDoMercadoRegular,TaxaDeDividendo,RendimentoDeDividendo,DataExDividendo,RelacaoDePagamento,RendimentoMedioDeDividendoDeCincoAnos,Beta,PeRetrospectivo,PeFuturo,VolumeMedio,VolumeMedioDe10Dias,VolumeDiarioMedioDe10Dias,Oferta,Pedido,CapitalizacaoDeMercado,AltaDe52Semanas,PrecoParaVendas12Meses,MediaDe50Dias,MediaDe200Dias,TaxaAnualDeDividendoRetrospectivo,RendimentoAnualDeDividendoRetrospectivo,Moeda,ValorDaEmpresa,MargensDeLucro,AcoesEmCirculacao,AcoesEmitidas,PercentualDetidoPorInsiders,PercentualDetidoPorInstituicoes,AcoesEmitidasImplicitas,ValorContabil,PrecoParaValorContabil,FimDoUltimoAnoFiscal,FimDoProximoAnoFiscal,UltimoTrimestre,CrescimentoTrimestralDosLucros,LucroLiquidoParaAcionistas,EpsRetrospectivo,EpsFuturo,RelacaoPeg,UltimoFatorDeDesdobramento,DataDoUltimoDesdobramento,ValorDaEmpresaParaReceitas,ValorDaEmpresaParaEbitda,MudancaDe52Semanas,MudancaDe52SemanasSandP,ValorDoUltimoDividendo,DataDoUltimoDividendo,Troca,TipoDeCotacao,Simbolo,SimboloSubjacente,NomeCurto,NomeCompleto,DataDaPrimeiraNegociacaoEpochUtc,NomeCompletoDoFusoHorario,NomeCurtoDoFusoHorario,Uuid,IdDoQuadroDeMensagens,OffsetGmtEmMilissegundos,PrecoAtual,PrecoAlvoMaximo,PrecoAlvoMinimo,PrecoAlvoMedio,MediaDasRecomendacoes,ChaveDasRecomendacoes,NumeroDeOpiniaoDosAnalistas,CaixaTotal,CaixaTotalPorAcao,Ebitda,DividaTotal,IndiceDeLiquidezImediata,IndiceDeLiquidezCorrente,ReceitaTotal,DividaParaPatrimonio,ReceitaPorAcao,RetornoSobreAtivos,RetornoSobrePatrimonio,FluxoDeCaixaLivre,FluxoDeCaixaOperacional,CrescimentoDosLucros,CrescimentoDaReceita,MargensBrutas,MargensEbitda,MargensOperacionais,MoedaFinanceira,RelacaoPegRetrospectiva,SMA_10,EMA_10,RSI,Williams_%R,ROC,MACD,MACD_signal,bollinger_hband,bollinger_lband,Percentual,Tendencia
202041,RPAD5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.829165,65.169568,NaN,NaN,0.093038,0.093038,NaN,NaN,0.000000,0
19260,CPLE6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.711997,64.445810,NaN,NaN,0.165117,0.165117,NaN,NaN,0.000000,0
211731,MTSA4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.699656,39.284237,NaN,NaN,-0.359417,-0.359417,NaN,NaN,0.000000,0
3431,B3SA3,2023-08-01,14.79,14.58,14.87,14.74,29131900.0,0.0,0.0,São Paulo,SP,Financial Data & Stock Exchanges,financial-data-stock-exchanges,Financial Data & Stock Exchanges,Financial Services,financial-services,Financial Services,NaN,3.0,1.0,7.0,5.0,1.0,1.725149e+09,86400.0,2.0,12.3,12.3,0.31,0.0249,

In [ ]:
# Remover valores nulos de data
dados_full.dropna(subset=['Data'], inplace=True)

In [ ]:
# Verificar Distribuição da Target
dados_full['Tendencia'].value_counts()

,count
Tendencia,
0,95537
-1,53465
1,50334


In [ ]:
# Estatísticas descritivas de dados numéricos
dados_full.describe()

,Abertura,MenorValor,MaiorValor,Fechamento,Volume,Dividendos,Desdobramento,EmpregadosEmTempoIntegral,RiscoDeAuditoria,RiscoDoConselho,RiscoDeCompensacao,RiscoDeDireitosDeAcionistas,RiscoGeral,DataDaEraDeGovernanca,IdadeMaxima,DicaDePreco,FechamentoAnterior,FechamentoAnteriorDoMercadoRegular,TaxaDeDividendo,RendimentoDeDividendo,DataExDividendo,RelacaoDePagamento,RendimentoMedioDeDividendoDeCincoAnos,Beta,PeRetrospectivo,PeFuturo,VolumeMedio,VolumeMedioDe10Dias,VolumeDiarioMedioDe10Dias,Oferta,Pedido,CapitalizacaoDeMercado,AltaDe52Semanas,PrecoParaVendas12Meses,MediaDe50Dias,MediaDe200Dias,TaxaAnualDeDividendoRetrospectivo,RendimentoAnualDeDividendoRetrospectivo,ValorDaEmpresa,MargensDeLucro,AcoesEmCirculacao,AcoesEmitidas,PercentualDetidoPorInsiders,PercentualDetidoPorInstituicoes,AcoesEmitidasImplicitas,ValorContabil,PrecoParaValorContabil,FimDoUltimoAnoFiscal,FimDoProximoAnoFiscal,UltimoTrimestre,CrescimentoTrimestralDosLucros,LucroLiquidoParaAcionistas,EpsRetrospectivo,EpsFuturo,RelacaoPeg,DataDoUltimoDesdobramento,ValorDaEmpresaParaReceitas,ValorDaEmpresaParaEbitda,MudancaDe52Semanas,MudancaDe52SemanasSandP,ValorDoUltimoDividendo,DataDoUltimoDividendo,DataDaPrimeiraNegociacaoEpochUtc,OffsetGmtEmMilissegundos,PrecoAtual,PrecoAlvoMaximo,PrecoAlvoMinimo,PrecoAlvoMedio,MediaDasRecomendacoes,NumeroDeOpiniaoDosAnalistas,CaixaTotal,CaixaTotalPorAcao,Ebitda,DividaTotal,IndiceDeLiquidezImediata,IndiceDeLiquidezCorrente,ReceitaTotal,DividaParaPatrimonio,ReceitaPorAcao,RetornoSobreAtivos,RetornoSobrePatrimonio,FluxoDeCaixaLivre,FluxoDeCaixaOperacional,CrescimentoDosLucros,CrescimentoDaReceita,MargensBrutas,MargensEbitda,MargensOperacionais,RelacaoPegRetrospectiva,SMA_10,EMA_10,RSI,Williams_%R,ROC,MACD,MACD_signal,bollinger_hband,bollinger_lband,Percentual,Tendencia
count,199336.000000,199336.000000,199336.000000,199336.000000,1.993360e+05,199336.000000,199336.000000,79107.000000,42444.000000,42444.000000,42444.000000,42444.000000,42444.000000,4.244400e+04,199336.0,199336.000000,199336.000000,199336.000000,141248.000000,141248.000000,1.753550e+05,124771.000000,99074.000000,197328.000000,146170.000000,116657.000000,1.993360e+05,1.993360e+05,1.993360e+05,197346.000000,197846.000000,1.973880e+05,199336.000000,194393.000000,199336.000000,199336.000000,92355.000000,92355.000000,1.963910e+05,191897.000000,1.952450e+05,1.963930e+05,179407.000000,176411.000000,1.496800e+04,198390.000000,179265.000000,1.983900e+05,1.983900e+05,1.983900e+05,125343.000000,1.978910e+05,196827.000000,116160.000000,74220.000000,1.204630e+05,195392.000000,177418.000000,15467.000000,15467.000000,13471.000000,1.347100e+04,1.993360e+05,199336.0,198887.000000,102182.000000,102182.000000,102182.000000,113663.000000,102182.000000,1.963910e+05,196391.000000,1.774180e+05,1.913980e+05,181913.000000,181913.000000,1.958920e+05,157797.000000,195892.000000,196394.000000,177769.000000,1.789190e+05,1.968920e+05,124844.000000,190652.000000,179416.000000,174770.000000,195892.000000,4494.000000,195745.000000,199327.000000,199323.000000,187552.000000,194548.000000,199311.000000,199303.000000,191755.000000,191755.000000,198937.000000,199336.000000
mean,19.338684,19.065900,19.609332,19.342064,2.947223e+06,0.000130,0.000007,14605.754535,5.341886,5.093983,4.976510,5.799642,5.188154,1.725149e+09,86400.0,2.462375,18.390795,18.390795,1.465293,0.066387,1.675578e+09,1.363005,5.051315,0.778397,19.291289,5.808836,2.612466e+06,2.741392e+06,2.741392e+06,17.918655,18.744073,1.787502e+10,25.077347,2.969453,18.479845,18.994417,1.268480,0.054848,3.191453e+10,0.980716,4.953109e+08,6.535469e+08,0.532016,0.281104,2.280788e+09,-18.419694,4.092341,1.703854e+09,1.735475e+09,1.719343e+09,0.978909,1.649435e+09,-213.475714,1.853567,1.131213,1.436598e+09,4.652406,13.172748,0.023197,0.229554,0.264882,1.675013e+09,1.214789e+09,-10800000.0,18.458024,27.338861,17.990755,22.199760,2.380374,6.976963,1.185847e+10,8.893282,3.418236e+09,2.549895e+10,1.877233,2.401165,1.583877e+10,132.478831,33.570915,0.036260,0.075384,1

In [ ]:
# Estatísticas descritivas de dados categóricos
dados_full.describe(exclude=np.number)

,Ticker,Data,Cidade,Estado,Industria,ChaveIndustria,DescricaoIndustria,Setor,ChaveSetor,DescricaoSetor,Moeda,UltimoFatorDeDesdobramento,Troca,TipoDeCotacao,Simbolo,SimboloSubjacente,NomeCurto,NomeCompleto,NomeCompletoDoFusoHorario,NomeCurtoDoFusoHorario,Uuid,IdDoQuadroDeMensagens,ChaveDasRecomendacoes,MoedaFinanceira
count,199336,199336,197890,196894,198390,15467,15467,198390,15467,15467,199336,120463,199336,199336,199336,199336,199336,199336,199336,199336,199336,15467,198887,198887
unique,399,501,79,17,91,26,26,11,8,8,1,56,1,1,399,399,397,315,1,1,399,30,5,1
top,IGTI11,2024-06-18,São Paulo,SP,Utilities—Regulated Electric,banks-regional,Banks - Regional,Consumer Cyclical,financial-services,Financial Services,BRL,2:1,SAO,EQUITY,IGTI11.SA,IGTI11.SA,MRS LOGISTICA,Iguatemi S.A.,America/Sao_Paulo,BRT,15120b59-d6a9-3442-b283-2a7464bd3c28,finmb_878163,none,BRL
freq,1000,402,78291,103761,16450,1995,1995,32959,3991,3991,199336,14477,199336,199336,1000,1000,1500,2743,199336,199336,1000,998,85224,198887


In [ ]:
# Copia do DataFrame Para Análise de Correlação (Para tratamento de Variáveis Categóricas)
analise_corr = dados_full.copy()

analise_corr.info()

# Como temos variáveis categóricas, para análise da correlação com o método Spearman, precisamos alterar para variáveis numéricas
# Seleciona apenas as colunas categóricas
categorical_columns = analise_corr.select_dtypes(include=['object']).columns

# Aplicar Label Encoding para todas as colunas categóricas
analise_corr[categorical_columns] = analise_corr[categorical_columns].apply(lambda col: col.astype('category').cat.codes)

# Spearman: avalia relações monotônicas lineares ou não entre variáveis contínuas ou ordinais.
# É calculada aplicando a medida do coeficiente de Pearson aplicado aos valores da ordem de duas variáveis.
# Não é sensível a assimetrias na distribuição.

analise_corr.corr(method='spearman').Fechamento.sort_values(ascending=False)

<class 'pandas.core.frame.DataFrame'>
Index: 199336 entries, 181016 to 139120
Columns: 124 entries, Ticker to Tendencia
dtypes: float64(99), int64(1), object(24)
memory usage: 190.1+ MB


,Fechamento
Fechamento,1.000000
EMA_10,0.998894
SMA_10,0.998462
MaiorValor,0.998383
MenorValor,0.998077
Abertura,0.997899
bollinger_hband,0.994295
bollinger_lband,0.992036
MediaDe200Dias,0.928110
AltaDe52Semanas,0.920782


### Utilizamos variáveis com alta correlação no método spearman. Apesar das variáveis selecionadas terem uma alta correlação o que pode indicar uma redundância nos dados selecionados, optamos por mantê-las porque faz mais sentido pelo contexto, são variáveis com importância no dataset de ações:

*   EMA_10	0.998894
*   SMA_10	0.998462
*   MaiorValor	0.998383
*   MenorValor	0.998077
*   Abertura	0.997899
*   bollinger_hband	0.994295
*   bollinger_lband	0.992036
*   MediaDe200Dias	0.928110
*   AltaDe52Semanas	0.920782
*   MediaDe50Dias 0.903140














In [ ]:
features = ['Abertura','MenorValor','MaiorValor',
            'EMA_10','SMA_10','bollinger_hband','bollinger_lband'
            ,'MediaDe200Dias','AltaDe52Semanas','MediaDe50Dias','Tendencia']

df = dados_full[features]

In [ ]:
null_counts = df.isna().sum()

# Exibir apenas as colunas que têm valores nulos
null_columns = null_counts[null_counts > 0]
print(null_columns)

EMA_10                9
SMA_10             3591
bollinger_hband    7581
bollinger_lband    7581
dtype: int64


In [ ]:
# Remover valores nulos
df.dropna(subset=features, inplace=True)

<ipython-input-88-37a744bc1bfd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=features, inplace=True)


In [ ]:
null_counts = df.isna().sum()

# Exibir apenas as colunas que têm valores nulos
null_columns = null_counts[null_counts > 0]
print(null_columns)

Series([], dtype: int64)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 191755 entries, 181244 to 139120
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Abertura         191755 non-null  float64
 1   MenorValor       191755 non-null  float64
 2   MaiorValor       191755 non-null  float64
 3   EMA_10           191755 non-null  float64
 4   SMA_10           191755 non-null  float64
 5   bollinger_hband  191755 non-null  float64
 6   bollinger_lband  191755 non-null  float64
 7   MediaDe200Dias   191755 non-null  float64
 8   AltaDe52Semanas  191755 non-null  float64
 9   MediaDe50Dias    191755 non-null  float64
 10  Tendencia        191755 non-null  int64  
dtypes: float64(10), int64(1)
memory usage: 17.6 MB


# **7 - Preparar os Dados Para Treino do Modelo**

In [ ]:
# Alterar Variável Tendência para Números Inteiros
df['Tendencia'] = df['Tendencia'].replace({-1: 0, 0: 1, 1: 2})

<ipython-input-117-b33574807287>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tendencia'] = df['Tendencia'].replace({-1: 0, 0: 1, 1: 2})


In [ ]:
# Separar Base Entre X e y
X = df[['Abertura','MenorValor','MaiorValor',
        'EMA_10','SMA_10','bollinger_hband','bollinger_lband'
        ,'MediaDe200Dias','AltaDe52Semanas','MediaDe50Dias']]

y = df['Tendencia']

In [ ]:
# Dividir em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **8 - Treinar e Testar Modelos**

## 8.1 - Random Forest

In [ ]:
# Inicializar o modelo
model = RandomForestClassifier(n_estimators=300, random_state=42)

# Treinando o modelo
model.fit(X_train, y_train)

# Prevendo nos dados de teste
y_pred = model.predict(X_test)

# Avaliando o modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy:.2f}")
print(classification_report(y_test, y_pred))


Acurácia: 0.62
              precision    recall  f1-score   support

           0       0.57      0.50      0.54     10312
           1       0.65      0.76      0.70     18292
           2       0.59      0.47      0.52      9747

    accuracy                           0.62     38351
   macro avg       0.60      0.58      0.59     38351
weighted avg       0.61      0.62      0.61     38351



## 8.2 - KNN

In [ ]:
# Criar o modelo KNN
knn_model = KNeighborsClassifier(n_neighbors=19)

# Treinar o modelo
knn_model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = knn_model.predict(X_test)

# Avaliar o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))

Acurácia: 0.5212380381215613
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.42      0.35      0.38     10312
           1       0.58      0.79      0.67     18292
           2       0.42      0.20      0.27      9747

    accuracy                           0.52     38351
   macro avg       0.47      0.45      0.44     38351
weighted avg       0.49      0.52      0.49     38351



## 8.3 - XGBoost


In [ ]:
#XGBOOST
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

# Avaliar o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:14:28] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Acurácia: 0.6511694610310031
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.64      0.53      0.58     10312
           1       0.64      0.82      0.72     18292
           2       0.69      0.47      0.56      9747

    accuracy                           0.65     38351
   macro avg       0.66      0.61      0.62     38351
weighted avg       0.66      0.65      0.64     38351



In [ ]:
#Aplicar a Padronização das Variáveis

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Supondo que seu array seja chamado de 'meu_array'
df2 = pd.DataFrame(X_train)

# Agora você pode usar o método head()
print(df.head())

        Abertura  MenorValor  MaiorValor     EMA_10  SMA_10  bollinger_hband  \
181244     21.17       20.97       21.30  21.002724  21.074        21.692526   
181272     21.14       20.80       21.14  20.989501  21.068        21.650929   
181284     20.89       20.70       20.99  20.951410  21.044        21.534193   
181296     20.64       20.18       20.72  20.882063  20.982        21.477405   
181307     20.72       20.28       20.87  20.879870  20.961        21.461139   

        bollinger_lband  MediaDe200Dias  AltaDe52Semanas  MediaDe50Dias  \
181244        19.762474        10.33465            17.32        12.9412   
181272        19.918071        10.33465            17.32        12.9412   
181284        20.147807        10.33465            17.32        12.9412   
181296        20.254595        10.33465            17.32        12.9412   
181307        20.318861        10.33465            17.32        12.9412   

        Tendencia  
181244          1  
181272          1  
181284  

In [ ]:
# Balanceamento
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)

In [ ]:
X_resampled.sample(3)

,Abertura,MenorValor,MaiorValor,EMA_10,SMA_10,bollinger_hband,bollinger_lband,MediaDe200Dias,AltaDe52Semanas,MediaDe50Dias
138791,14.110000,13.080000,14.110000,13.174337,13.332000,14.374204,10.507796,11.64990,14.60,11.6194
199930,4.622478,4.470255,4.624359,4.823565,4.908726,5.390394,4.504409,5.13460,7.62,5.6480
35289,10.340000,9.930000,10.340000,10.620565,10.657000,12.579428,9.816572,12.76615,15.73,11.9606


In [ ]:
y_resampled.value_counts()

,count
Tendencia,
1,91937
0,91937
2,91937


In [ ]:
# Dividir em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

## 8.4 - Random Forest (Balanceado e Padronizado)

In [ ]:
# Inicializar o modelo
model = RandomForestClassifier(n_estimators=300, random_state=42)

# Treinando o modelo
model.fit(X_train, y_train)

# Prevendo nos dados de teste
y_pred = model.predict(X_test)

# Avaliando o modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy:.2f}")
print(classification_report(y_test, y_pred))


Acurácia: 0.75
              precision    recall  f1-score   support

           0       0.75      0.80      0.77     18321
           1       0.72      0.65      0.68     18223
           2       0.77      0.80      0.78     18619

    accuracy                           0.75     55163
   macro avg       0.75      0.75      0.75     55163
weighted avg       0.75      0.75      0.75     55163



## 8.5 - KNN (Balanceado e Padronizado)

In [ ]:
# Criar o modelo KNN
knn_model = KNeighborsClassifier(n_neighbors=3)

# Treinar o modelo
knn_model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = knn_model.predict(X_test)

# Avaliar o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))

Acurácia: 0.6235338904700615
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.58      0.70      0.63     18321
           1       0.65      0.56      0.60     18223
           2       0.65      0.62      0.63     18619

    accuracy                           0.62     55163
   macro avg       0.63      0.62      0.62     55163
weighted avg       0.63      0.62      0.62     55163



## 8.6 - XGBoost (Balanceado e Padronizado)

In [ ]:
#XGBOOST
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

# Avaliar o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:53:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Acurácia: 0.6759059514529666
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.68      0.72      0.70     18321
           1       0.63      0.62      0.63     18223
           2       0.72      0.68      0.70     18619

    accuracy                           0.68     55163
   macro avg       0.68      0.68      0.68     55163
weighted avg       0.68      0.68      0.68     55163

